In [347]:
import os,json

def createdir(dirName):
    try:
        os.mkdir(dirName)
        print("Directory " , dirName ,  " Created ") 
    except FileExistsError:
        print("Directory " , dirName ,  " already exists")
        
#write inv/txt file
orpath = "C:\\Users\\rinat\\Desktop\\now\\russian\\"
fname = 'invert_ind_table.txt'

In [348]:


#if os.path.isfile(orpath+fname):
mypath = r"C:\Users\rinat\Desktop\now\russian\resultlem"
createdir(mypath)
values = os.listdir(mypath)
dic = {}
arr = []
dic['sumallwords'] = {}
for f in values:
    x = 0
    with open(os.path.join(mypath, f), "r", encoding='utf-8')  as f1:
        for line in f1:
            for word in line.split():
                if word not in dic:
                    dic[word] = {}
                if f not in dic[word]:
                    dic[word][f] = [x]
                else:
                    dic[word][f].append(x)
                x+=1
    dic['sumallwords'][f] = x
for word in dic.keys():
    if word != "sumallwords":
        for href in dic[word].keys():
            if href != 'idf':
                st = dic[word][href]
                if type(st) != int:
                    dic[word][href] = len(dic[word][href])
                else:
                    dic[word][href] = 1
with open(fname, 'w', encoding='utf-8', ) as file:
     file.write(json.dumps(dic, ensure_ascii=False)) # use `json.loads` to do the reverse
len(dic)

Directory  C:\Users\rinat\Desktop\now\russian\resultlem  already exists


25241

In [349]:
#reading created file above
dicton = None
if os.path.isfile(orpath+fname) != False:
    with open(os.path.join(orpath, fname), "r", encoding='utf-8')  as f1:
        dicton = json.load(f1)
print(len(dicton))

25241


In [350]:
#parse input
import re

def parseStrInput(value):
    if len(value) > 100 and len(value) == 0:
        err = "length incorrect"
        raise err
    value = re.sub('[!,.?]', '', value)
    
    value = value.replace(' не ', ' ¬ ')
    value = value.replace(' или ', ' | ')
    value = value.replace(' и ', " & ")
    value = value.replace(' не ', ' ¬ ')
    value = value.replace(' или ', ' | ')
    value = value.replace(' и ', " & ")
    
    exc = str(value[0]+value[1]+value[2])
    if exc == 'не ':
        value = value.replace('не ', '¬ ', 1)
    size = len(value)    
    exc = str(value[size-1]+value[size-2]+value[size-3])
    if exc == 'ен ':
        value = value.replace(' не', '', -1)   
        
    value = " ".join(value.split())
    condlist = ['| |', '& &', '¬ ¬', '| &','& |','¬ | ¬', '| ¬ |','¬ & ¬', '& ¬ &']
    replist = ['|', '&', '¬', '|', '&', '| ¬','| ¬', '& ¬', '& ¬']
    
    for i in range(len(condlist)):
        cond = condlist[i]
        val = replist[i]
        while cond in value:
            value = value.replace(cond, val, 1)   
    for i in range(len(condlist)):
        cond = condlist[i]
        val = replist[i]
        while cond in value:
            value = value.replace(cond, val, 1) 
    return parseOprsInput(value)

def parseOprsInput(value):
    exclist = {'|','&'}
    pvalue = value.split(" ")
    if pvalue[0] in exclist:
        pvalue = pvalue[1:]
    if pvalue[-1] in exclist:
        pvalue = pvalue[0:-1]
        
    if len(pvalue) > 100 and len(pvalue) == 0:
        err = "length incorrect"
        raise err     
    parsed = pvalue
    fexclist = {'|','&','¬'}
    text = []
    oprs = []
    ind = 0
    k = 0
    cur_word = None
    if parsed[0] == "¬":
        oprs.append(["¬", 0])
        text.append(parsed[1])
        ind=1
        k = 2
    for i in range(k,len(parsed)):
        val = parsed[i]
        if val in fexclist:
            oprs.append([val, ind])
        else:     
            if i - 1 >= 0 and i < len(parsed):
                if parsed[i-1] == '¬':
                    if parsed[i-2] not in exclist and val not in exclist:
                        oprs.append(['&', ind])
                else:
                    if parsed[i-1] not in exclist and val not in exclist:
                        oprs.append(['&', ind])
            ind += 1
            text.append(val)
    dic = {}
    for i in range(len(text)):
        key = text[i]
        dic[key] = {}
        opp = []
        for op in oprs:
            if op[1] == i:
                dic[key][op[0]] = ''
    return dic

In [351]:
def search(value,oper, dictonary):
    notf = 'combination not found or one of these words is missing!'
    notfone = 'one of these words is missing!'
    failedstr = 'value error!'
    message = []
    failed = False
    result = {}
    count = 0
    dic = dictonary
    words = value.split(' ')
    for i in range(len(words)):
        words[i] = morph.parse(words[i])[0].normal_form
        
    if str(value.lower()) != "" and str(value.lower()) != " " and str(value.lower()) != None:
        for word in words:
            if word in dic:
                if len(word) != 0:
                    result = result
                    result[word] = dic[word]
                    count+=1
                    continue
    else:
        return failedstr
    
    if count != len(words):
        failed = True
        message.append(notfone)
   
    if str(oper.lower()) == "and" and failed != True:
        keys = list(result.keys())
        res = set(result[keys[0]].keys())
        for i in range(len(keys)):
            if i != 0:
                res = res & set(result[keys[i]].keys())
        return [message,res]
            
        
    
    if str(oper.lower()) == "or" or failed == True:
        if len(result) != 0: 
            keys = []
            for word in result.keys():
                for key in result[word]:
                    keys.append(key)
            if failed != True:
                return keys
                #return list(dict.fromkeys(keys))
            else:
                return [notf, keys]
                #return [notf, list(dict.fromkeys(keys))]

In [445]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def dict_compare(d1, d2):
    dict_a = d1
    dict_b = d2
    keys_a = set(dict_a.keys())
    keys_b = set(dict_b.keys())
    keys = keys_a & keys_b
    new_res = d1
    res = {}
    for key in keys:
        val = new_res[key]
        res[key] = val
    return res
def inter(res,local_res):
    if len(res) != 0 and len(local_res) != 0:
        if len(res)<=len(local_res): 
            res = dict_compare(res, local_res)
        else:
            res = dict_compare(local_res, res)
    else:
        res = {}
    return res
def union(res, local_res):
    if len(res) != 0 and len(local_res) != 0:
        res = {**res, **local_res}
    else:
        if len(res) == 0 and len(local_res) == 0:
            res = {}
        else:
            if len(local_res) == 0:
                res = local_res
            else:
                res = res
    return res

def simple_search(value, dic):
    notfound = 'not found!'
    value = str(value.lower())
    value = morph.parse(value)[0].normal_form
    res = {}
    if value in dic:
        return dic[value]
    else:
        return {}

def dicDif(value, dic):
    notfound = 'not found!'
    value = str(value.lower())
    value = morph.parse(value)[0].normal_form
    res = {}
    local = {}
    if value in dic:
        for href,val in dic[value].items():
            local[href] = val
            
    for word in dic:
        if word != value and word != 'sumallwords':
            hrefs = {}
            for href, val in dic[word].items():
                if href not in local:
                    hrefs[href] = val
            if len(res) != 0 and len(hrefs) != 0:
                res = {**res, **hrefs}
            if len(res) == 0:
                res = hrefs
                
            if len(hrefs) == 0:
                continue
    return res

def main_search(value, dictonary):
    res = {}
    if len(value) == 0:
        print('oops')
        return None
    #print(value)
    if len(value) == 1:
        for key,val in value.items():
            key = morph.parse(key)[0].normal_form
            key = key.lower()
            if len(val) != 0:
                if '¬' in val.keys():
                    return dicDif(key,dictonary)
                else:
                    return simple_search(key,dictonary)

    local_res = {}
    for key,val in value.items():
        key = morph.parse(key)[0].normal_form
        #print(key,val)
        if '¬' in val.keys():
            local_res = dicDif(key,dictonary)
        else:
            local_res = simple_search(key,dictonary)
        if '&' not in val.keys() and '|' not in val.keys():
            res = local_res
            #print(len(res))
            continue
        else:
            #print(len(local_res))
            if '&' in val.keys() :
                res = inter(res, local_res)
            else:
                if '|' in val.keys():
                    res = union(res, value)
    return res

value = 'нервы и деньги и время'
dic = parseStrInput(value)
res = main_search(dic, dicton)
res

{'lemmet39_page_Треугольни.txt': 11}

In [446]:
value = 'зачем понаписать ерунда вы же сказать что драйвер должный быть простой понятный руководство'
dic = parseStrInput(value)
res = main_search(dic, dicton)
res

{'lemmet44_page_Как новые .txt': 1}